In [1]:
import pandas as pd
import numpy as np
import glob
import scipy.stats as stats

In [2]:
featureset=[
'acci_last_infractionage',
'carpoolind_encd',
'classcd_encd',
'driverage',
'drivernumber',
'estimatedannualdistance',
'gooddriverind_encd',
'maritalstatuscd_encd',
'mvrstatus_encd',
'mvrstatusage',
'ratingvalue',
'vehbodytypecd_encd',
'vehicleage',
'vehnumber',
'licenseage',
'gendercd_encd',
'external_length_in',
'external_width_in'
]

need_refline_set=['driverage', 
                  'drivernumber', 
                  'estimatedannualdistance', 
                  'ratingvalue',
                  'vehicleage',
                  'vehnumber',
                  'licenseage',
                  'external_length_in',
                  'external_width_in']

In [3]:
significance_level=0.00001

In [4]:
#models tables files dir
ModelsTblDir='/home/kate/Models/XGB/tbl/'

In [5]:
#partial dependency
for f in featureset:
    print (f)
    PDFList=glob.glob('%sAll*PD*%s.csv'%(ModelsTblDir,f))
    i=0
    Models=[]
    for t in PDFList:
        print(t)
        ModelName=t[33:40]
        print(ModelName)
        Models.append(ModelName)
        df = pd.read_csv(t, index_col=None)
        if i==0:
            if '_encd' in f:
                pdf_df=df[[f,f.replace('_encd',''),'hasclaim','count','pd']]
                pdf_df.columns=[f,f.replace('_encd',''),'hasclaim','count',ModelName]
            elif f in ['estimatedannualdistance','ratingvalue']:
                pdf_df=df[[f,'bin','hasclaim','count','pd']]
                pdf_df.columns=[f,'bin','hasclaim','count',ModelName]                
            else:
                pdf_df=df[[f,'hasclaim','count','pd']]
                pdf_df.columns=[f,'hasclaim','count',ModelName]
        else:
            pdf_df=pd.concat([pdf_df,df[['pd']]], axis=1) 
            pdf_df.rename(columns={'pd':ModelName}, inplace=True)
        i=+1
    pdf_df['Partial Dependency'] = pdf_df[Models].mean(axis=1)
    if f in need_refline_set:
        pdf_df.sort_values(f,inplace=True)
        pdf_df.reset_index(inplace=True)
        pdf_df['pvalue']= np.nan
        pdf_df['statistic']= np.nan
        pdf_df['isRefLine']= np.nan
        for index in range(1,len(pdf_df)):
            row=pdf_df.iloc[index][['Model 2','Model 6','Model 1','Model 5','Model 8','Model 3','Model 0','Model 9','Model 7','Model 4']]
            if index==1:
                RefLine=row
                pdf_df.at[index,'isRefLine']=0
                pdf_df.at[index,'pvalue']=0
                pdf_df.at[index,'statistic']=0
            else:
                t=stats.ttest_ind(RefLine.tolist(),row.tolist())
                pdf_df.at[index,'pvalue']=t.pvalue
                pdf_df.at[index,'statistic']=t.statistic
                if t.pvalue<=0.00001:
                    RefLine=row
                    pdf_df.at[index,'isRefLine']=0.025
                else:
                    pdf_df.at[index,'isRefLine']=0
    pdf_df.to_csv(ModelsTblDir+'PD_%s_All.csv'%f)

acci_last_infractionage
/home/kate/Models/XGB/tbl/All PD Model 2 acci_last_infractionage.csv
Model 2
/home/kate/Models/XGB/tbl/All PD Model 4 acci_last_infractionage.csv
Model 4
/home/kate/Models/XGB/tbl/All PD Model 3 acci_last_infractionage.csv
Model 3
/home/kate/Models/XGB/tbl/All PD Model 8 acci_last_infractionage.csv
Model 8
/home/kate/Models/XGB/tbl/All PD Model 1 acci_last_infractionage.csv
Model 1
/home/kate/Models/XGB/tbl/All PD Model 9 acci_last_infractionage.csv
Model 9
/home/kate/Models/XGB/tbl/All PD Model 7 acci_last_infractionage.csv
Model 7
/home/kate/Models/XGB/tbl/All PD Model 5 acci_last_infractionage.csv
Model 5
/home/kate/Models/XGB/tbl/All PD Model 6 acci_last_infractionage.csv
Model 6
/home/kate/Models/XGB/tbl/All PD Model 0 acci_last_infractionage.csv
Model 0
carpoolind_encd
/home/kate/Models/XGB/tbl/All PD Model 1 carpoolind_encd.csv
Model 1
/home/kate/Models/XGB/tbl/All PD Model 8 carpoolind_encd.csv
Model 8
/home/kate/Models/XGB/tbl/All PD Model 6 carpoolind_

/home/kate/Models/XGB/tbl/All PD Model 3 vehbodytypecd_encd.csv
Model 3
/home/kate/Models/XGB/tbl/All PD Model 8 vehbodytypecd_encd.csv
Model 8
/home/kate/Models/XGB/tbl/All PD Model 6 vehbodytypecd_encd.csv
Model 6
/home/kate/Models/XGB/tbl/All PD Model 1 vehbodytypecd_encd.csv
Model 1
vehicleage
/home/kate/Models/XGB/tbl/All PD Model 2 vehicleage.csv
Model 2
/home/kate/Models/XGB/tbl/All PD Model 3 vehicleage.csv
Model 3
/home/kate/Models/XGB/tbl/All PD Model 9 vehicleage.csv
Model 9
/home/kate/Models/XGB/tbl/All PD Model 6 vehicleage.csv
Model 6
/home/kate/Models/XGB/tbl/All PD Model 7 vehicleage.csv
Model 7
/home/kate/Models/XGB/tbl/All PD Model 5 vehicleage.csv
Model 5
/home/kate/Models/XGB/tbl/All PD Model 4 vehicleage.csv
Model 4
/home/kate/Models/XGB/tbl/All PD Model 8 vehicleage.csv
Model 8
/home/kate/Models/XGB/tbl/All PD Model 1 vehicleage.csv
Model 1
/home/kate/Models/XGB/tbl/All PD Model 0 vehicleage.csv
Model 0
vehnumber
/home/kate/Models/XGB/tbl/All PD Model 8 vehnumber.

In [6]:
#Feature Importance
FEList=glob.glob('%sAll Feature Importance*.csv'%ModelsTblDir)
Models=[]
fe_df=pd.DataFrame()
i=0;
for t in FEList:
    print(t)
    ModelName=t[49:56]
    print(ModelName)
    df = pd.read_csv(t, index_col=None,header=None)
    df.columns=['Feature',ModelName]
    df['Feature']=pd.to_numeric(df['Feature'].str.extract('(\d+)'))
    df.sort_values(by='Feature',inplace=True)
    if i>0:
        fe_df=pd.merge(fe_df,df, on='Feature')
    else:
        fe_df=df
    i=+1

/home/kate/Models/XGB/tbl/All Feature Importance Model 2.csv
Model 2
/home/kate/Models/XGB/tbl/All Feature Importance Model 3.csv
Model 3
/home/kate/Models/XGB/tbl/All Feature Importance Model 6.csv
Model 6
/home/kate/Models/XGB/tbl/All Feature Importance Model 5.csv
Model 5
/home/kate/Models/XGB/tbl/All Feature Importance Model 1.csv
Model 1
/home/kate/Models/XGB/tbl/All Feature Importance Model 4.csv
Model 4
/home/kate/Models/XGB/tbl/All Feature Importance Model 0.csv
Model 0
/home/kate/Models/XGB/tbl/All Feature Importance Model 8.csv
Model 8
/home/kate/Models/XGB/tbl/All Feature Importance Model 7.csv
Model 7
/home/kate/Models/XGB/tbl/All Feature Importance Model 9.csv
Model 9


/home/kate/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if sys.path[0] == '':


In [7]:
def GetFeatureName(i):
    return featureset[int(i)].replace('_encd','')
fe_df['Feature'] = fe_df.apply(lambda row: GetFeatureName(row['Feature']), axis=1)

In [8]:
fe_df['Importance']=fe_df.mean(axis=1)

In [9]:
fe_df.to_csv(ModelsTblDir+'FeatureImportance_All.csv')